# Continuous Control

---

## Import libraries

In [13]:
import torch
from unityagents import UnityEnvironment
import numpy as np
from collections import deque
import matplotlib.pyplot as plt
%matplotlib inline

from ddpg_agent import Agent

### 1. Start the Environment


In [14]:
env = UnityEnvironment(file_name='/mnt/store_data/deep-reinforcement-learning/p2_continuous-control/Reacher_Linux/Reacher.x86')

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_speed -> 1.0
		goal_size -> 5.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [15]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

In this environment, a double-jointed arm can move to target locations. A reward of `+0.1` is provided for each step that the agent's hand is in the goal location. Thus, the goal of your agent is to maintain its position at the target location for as many time steps as possible.

The observation space consists of `33` variables corresponding to position, rotation, velocity, and angular velocities of the arm.  Each action is a vector with four numbers, corresponding to torque applicable to two joints.  Every entry in the action vector must be a number between `-1` and `1`.

Run the code cell below to print some information about the environment.

In [21]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

### 3. Train an Agent in the Environment

In [ ]:
# env.seed(10)
random_seed = 10
agent = Agent(state_size=state_size, action_size=action_size, random_seed=random_seed)

NameError: name 'state_size' is not defined

In [ ]:
def ddpg(agent, brain_name, n_episodes=2000, max_t=1000, window_size=100, score_threshold=30.0, print_interval=10):
    """
    Deep Deterministic Policy Gradient (DDPG) training loop.

    Args:
    - agent: the DDPG agent
    - brain_name: the name of the brain associated with the environment
    - n_episodes: maximum number of training episodes (default=2000)
    - max_t: maximum number of timesteps per episode (default=1000)
    - window_size: size of the moving window to calculate the average score (default=100)
    - score_threshold: the average score threshold for considering the environment solved (default=30.0)
    - print_interval: interval for printing the training progress (default=10)

    Returns:
    - scores: list of scores obtained during training

    This function performs the DDPG training loop for the given agent and environment.
    It trains the agent for a maximum number of episodes, with a maximum number of timesteps per episode.
    The training progress is printed at regular intervals, and the function returns the list of scores obtained during training.
    """
    # Initialize variables
    scores_deque = deque(maxlen=window_size) # A deque is a double-ended queue that allows efficient appending and popping of elements
    scores = [] # List to store the scores obtained during training
    max_score = -np.inf # Initialize the maximum score seen so far to negative infinity
    
    # Loop over episodes
    for i_episode in range(1, n_episodes+1):
        
        # Reset the environment and get the initial state
        env_info = env.reset(train_mode=True)[brain_name]
        states = env_info.vector_observations
        
        # Reset the agent and initialize the episode score
        agent.reset()
        episode_scores = np.zeros(num_agents)
        
        # Loop over timesteps within the episode
        for _ in range(max_t):
            # Choose an action based on the current state
            actions = agent.act(states)
            # Take a step in the environment based on the chosen action
            env_info = env.step(actions)[brain_name]
            # Get the next state, reward, and done flag
            next_states = env_info.vector_observations
            rewards = env_info.rewards
            dones = env_info.local_done

            # Update the agent based on the experience tuple
            agent.step(states=states, actions=actions, rewards=rewards, next_states=next_states, dones=dones)
            # Update the episode score and current state
            episode_scores += np.array(rewards)
            states = next_states
            # Exit the loop if done
            if np.any(dones):
                break

        # Calculate the episode score and update the scores lists
        episode_score = np.mean(episode_scores)
        scores_deque.append(episode_score)
        scores.append(episode_score)
        average_score = np.mean(scores_deque)

        # Print the training progress at regular intervals
        print('\rEpisode: {}\tAverage Score: {:.2f}\tCurrent Score: {:.2f}'.format(i_episode, average_score, episode_score), end="")
        if i_episode % print_interval == 0:
            print('\rEpisode: {}\tAverage Score: {:.2f}\tCurrent Score: {:.2f}'.format(i_episode, average_score, episode_score))

        # Save the agent's weights if the environment is considered solved
        if average_score >= score_threshold:
            print('\nEnvironment solved in {} episodes!\tAverage Score: {:.2f}'.format(i_episode-window_size, average_score))
            torch.save(agent.actor_local.state_dict(), 'checkpoint_actor.pth')
            torch.save(agent.critic_local.state_dict(), 'checkpoint_critic.pth')
            break

    # Save the scores and close the environment
    np.save('scores.npy', scores)
    env.close()
    
    # Return the scores obtained during training
    return scores

In [ ]:
# Start training the DDPG agent for a maximum number of episodes, with a maximum number of timesteps per episode.

scores = ddpg(agent, brain_name, n_episodes=2000, max_t = 1000, window_size=100, score_threshold=30.0, print_interval=10)

NameError: name 'agent' is not defined

In [ ]:
# Create a figure and axis for plotting
fig = plt.figure()
ax = fig.add_subplot(111)

# Plot the scores over episodes
plt.plot(np.arange(len(scores)), scores)

# Set labels for the axes
plt.ylabel('Score')
plt.xlabel('Episode #')

# Save the plot as an image file
plt.savefig('ddpg_scores.png', bbox_inches='tight')

# Show the plot
plt.show()